In [1]:
from selenium import webdriver
from time import sleep
import threading
import re 
import urllib 
from bs4 import BeautifulSoup

In [67]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
items = pd.read_csv('data/item.csv')
items_detail= pd.read_csv('data/item_detail.csv')

In [3]:
len(items), len(items_detail)

(5180, 5180)

In [4]:
merged = pd.merge(items, items_detail, how='inner', on='id_item')

In [5]:
def getname(name, brand): 
    return name.replace(str(brand), '')

In [6]:
merged['item_name'] = merged.apply(lambda row: getname(row['name'], row['brand']), axis=1)

In [175]:
def get_comment(id_item, keyword, driver):
     
    ukey = urllib.parse.quote(keyword.encode('utf8'), '/:') 
    
    url = "https://msearch.shopping.naver.com/search/all.nhn?origQuery={}&pagingIndex=1&viewType=lst&sort=rel&showFilter=true&frm=NVSHSRC&selectedFilterTab=price&query={}".format(ukey,ukey)
       
    driver.get(url) 
    sleep(1)
     
    reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .grade') 
    rows = []
    if len(reviews) > 0: 
        for review in reviews:
            
            parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
            link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
            num = review.find_element_by_css_selector('.num').text
            cmts = review.find_element_by_css_selector('.help').text
            
            rows.append([id_item, link, num, cmts, keyword])
    else :
        reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .piece') 
        
        revcnt = 0
        
        for review in reviews:
            if review.text.startswith('리뷰'): 
                cmts = review.text.split('리뷰')[1] 
                revcnt += 1 
                parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
                link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
                #print(link)
                rows.append([id_item, link, np.nan, cmts, keyword])
        
        if revcnt == 0:
            rows.append([id_item, np.nan, np.nan, np.nan, keyword])
    return rows
    
def get_comment_group(items, names):
    driver =  webdriver.PhantomJS()
    for item, name in zip(items, names): 
         
        review_links = get_comment(item, name, driver)
        items_comments.extend(review_links)
    driver.close() 

In [176]:
!killall phantomjs

No matching processes belonging to you were found


In [177]:
items_comments = []
for cnt in range(1, 11): 
    start = 518 * (cnt-1)
    end = 518 * cnt-1
    items = merged.loc[start:end]['id_item'].values
    f_names = merged.loc[start:end]['fullname'].values
  
    th = threading.Thread(target=get_comment_group, args=(items,f_names, ))
    th.start()

In [197]:
review_info = pd.DataFrame(items_comments,columns=['id_item','link','point','cnt','keyword'])

In [171]:
!ls data

item.csv        item_detail.csv review_info.csv


In [200]:
#review_info.to_csv('data/review_info.csv',index=False)

In [7]:
review_info = pd.read_csv('data/review_info.csv', thousands=',')

In [8]:
n_review_info = review_info[review_info['point'].notna()]
n_review_info = n_review_info.reset_index(drop=True)
n_review_info['cnt'] = n_review_info['cnt'].astype(int)

In [9]:
len(n_review_info)

2471

In [155]:
def get_naver_review(url, cnt, iid, driver):
    #print(url)
    try:
        driver.get(url) 
        lis = driver.find_elements_by_css_selector('ul.lst_review li:not(.u_likeit_list)')
        sleep(2)

        #print(len(lis), cnt)
        
        # 추가정보 가져오기
        item_infos = driver.find_elements_by_css_selector('div.info_inner.list span')
        item_info=[]
       
        
        lp = ''
        lowestp = driver.find_elements_by_css_selector('div.price_area .low_price .num')
        if len(lowestp)> 0: 
            lp = lowestp[0].text
        
        item_info.append(iid)
        item_info.append(lp)
        for iinfo in item_infos:
            if len(iinfo.text)>0:
                item_info.append(iinfo.text)
 

            
        oricnt = cnt
        rows = [] 
        isone = False
        if oricnt < 20: 
            isone=True
            
        for page_seq in range(cnt//20 + 1): 
            for li in lis:  
                html= li.get_attribute('innerHTML')
                soup = BeautifulSoup(html) 
                
                atc_area = soup.find(class_="atc_area")
                avg_area = atc_area.find(class_='avg_area')
                
                point = avg_area.find(class_='curr_avg').get_text()
                site = avg_area.find(class_='path').get_text()
                person = avg_area.find(class_='name').get_text()

                date = avg_area.find(class_='date').get_text()
                subj = atc_area.find(class_='subjcet').get_text() 
                atc = atc_area.find(class_='atc').get_text()
                
                img = ''
                thmb = atc_area.find(class_='thmb')
                if thmb: 
                    img = thmb.find('img', class_='_reviewImg')['data-original']
                      
                rows.append([iid, point, site, person, date, subj, atc, img]) 
            
            
            if isone==False:
                # 페이지 갱신
                driver.execute_script("shop.detail.ReviewHandler.page({}, '_review_paging');".format(page_seq+2))
                sleep(0.5)
                lis = driver.find_elements_by_css_selector('ul.lst_review li:not(.u_likeit_list)')
 
    except Exception as e:
        print(iid, len(rows), oricnt)
        print(e) 
        print('========================')
      
        
    return rows, item_info


def get_naver_review_group(links, cnts, ids):
    
    driver =  webdriver.PhantomJS()
    for link, cnt, iid in zip(links, cnts, ids):  
        naver_reviews, item_info= get_naver_review(link, cnt, iid, driver)
        all_naver_reviews.extend(naver_reviews)
        all_naver_iteminfos.append(item_info)
    driver.close() 

In [170]:
!killall phantomjs

No matching processes belonging to you were found


In [144]:
driver =  webdriver.PhantomJS()

In [145]:
error = n_review_info[n_review_info.id_item==9708]

In [146]:
res = get_naver_review(error.link.values[0], error.cnt.values[0], error.id_item.values[0], driver)
res

http://cr2.shopping.naver.com/adcr.nhn?x=H%2FOitr82YlsfBccsU41P4%2F%2F%2F%2Fw%3D%3DsFJcGNh0AF7iR3wZKcZalvDjTQMT7EbYOt2cOHNYXMmPqozfnpIIP3ViBCUqbFFsvPbXj4Y4J6ss8KhBkRi30DLhQT%2BW4gc0uNitlN1j6kA6OWv8%2FgUKh3kWvU6sPBcxXg9YtPDnp3MjVED49YvLPRUcfz3X6oXfHTgsu4zAyaKMljutHGBy1qIHodf%2FQLs45DUdASP%2F4lCfXb9ApA5WU5E9dpE%2FDkkBDOxg008T8Yq%2Bfokx%2Bl4FoGy1QI9MhYsm97ezzdZ5SmNRKo6EhCioDiKLMX%2Fn%2FZ%2Bhq%2FAIysasXV1Y4X%2FiZpDKTXY19uS2ViNdn2rB5vlq29n47vtWdjAl4cBiH%2Bw0euBs3ivIRj69o8aHIxvF%2FUbTe%2BdWY6YwAWX%2FLrsGr0jnfGyRpQMPcD8jFqRFX%2F2bhCsptq%2B4ZJzYPv0JfIOZBUK32DfUes346tR0kVL7NZLfdN4POXF3tHaejBGNxbGLmO416bKJhNx38QFEbQNAn7EvVCiiznEAg4A6KzLHTz0rm5P1fwYogaxpHRwLJ3MPHpEgPZYf3ZKsnxEKpei8o85zv7B9D37qqN1%2B%2BfOJL93eaXr53ufr%2FnRJ7cwX4Y2199gcDtbO7VfdoT6WNuvTOpCTB3oxSVpw5s%2B4psckxhEJgbRvKN6pAwj%2BW3y9AzZ8s4O6uwJwOZOoX5zcIf9HVRGtjIjnuC03OXSMVl6OEbt1mTBMMhteOarcmZpSfm6jCWrA5SOOLQoktvGF9YoXg2c0%2FuewxLYHh6JOa3g0adOFDoLSQATzGQ2l%2BvA3%2BcwCbzph93wjplg5H12yaMzy4%2BXgIZZ6GtDP1qd6cVnbzs8aWVZtvzZh3mt8hXMM0OoenLB

KeyboardInterrupt: 

In [187]:
!killall phantomjs

No matching processes belonging to you were found


In [188]:
2471/ 3

823.6666666666666

In [189]:
all_naver_reviews = []
all_naver_iteminfos = []
for cnt in range(1, 4): 
    start = 823 * (cnt-1)
    end = 823 * cnt
    
    if cnt != 3:
        links = n_review_info.loc[start:end]['link'].values
        cnts = n_review_info.loc[start:end]['cnt'].values
        ids = n_review_info.loc[start:end]['id_item'].values 
    else :
        links = n_review_info.loc[start:]['link'].values
        cnts = n_review_info.loc[start:]['cnt'].values
        ids = n_review_info.loc[start:]['id_item'].values 
        print('end')
    
    print(start, end)
    th = threading.Thread(target=get_naver_review_group, args=(links, cnts, ids ))
    th.start()

0 823
823 1646
end
1646 2469


In [233]:
error_ids = [32997,28248,31333,29008,23710,13782,31165,22572,26857,22967,29941,31699,32904,32637,26826,28456,32681,27760,32680,30392,28859,28856,17359,25071,31914,32540,32974,31148,28428,28428,30959,17211,32537,32969,30957,29923,29994,29563,26990,27255,31878,22006,27926,32412,27822,23733,23732,26760,13297,27079,31725,32404,28001,26185,23314,25839,28162,27071,28271,27857,31779,15144,20364,25587,33179,28153 ]

In [237]:
complete_reviews = sample_reviews[~sample_reviews.id_item.isin(error_ids)]

In [242]:
notfinish = n_review_info[~n_review_info.id_item.isin(complete_reviews.id_item)]

In [244]:
notfinish.to_csv('data/notfinish.csv',index=False)

In [245]:
complete_reviews.to_csv('data/complete_reviews.csv',index=False)

In [ ]:
# 에러 다시 시작
# 안한거 다시 시작

In [221]:
len(all_naver_iteminfos), len(all_naver_reviews)

(1877, 2289569)

In [222]:
len(set(pd.DataFrame(all_naver_iteminfos)[0]))

1252

In [202]:
len(set(n_review_info.id_item))

1650

In [223]:
sample_reviews= pd.DataFrame(all_naver_reviews, columns=['id_item','point','site','author','date','sub','contents','img'])
sample_reviews

,id_item,point,site,author,date,sub,contents,img
0,28834,5,G마켓,pbr9****님,2014.12.23.,넘이뿌고 화려하고 빨리 건조되어 좋네요.,50대 후반인데 발라보니 넘 이뿌네요.색상이 화사하고 빨리 건조되어서 더욱더 좋네요...,
1,28834,5,GSSHOP,cjsy****님,2016.04.21.,색이은은해요..생각보다 반짝이는 잘 발리지않아요~~,색이은은해요..생각보다 반짝이는 잘 발리지않아요~~,
2,28834,4,SSG닷컴,swee****님,2016.03.26.,색상은 둘이 잘 어울리는데요. ?,색상은 둘이 잘 어울리는데요. 손톱에 손상을 많이 주는군요,
3,28834,4,신세계몰,swee****님,2016.03.26.,손톱에 손상을 많이 주는군요,손톱에 손상을 많이 주는군요,
4,28834,5,롯데닷컴,rmsc****님,2016.03.21.,색이 참 예뻐요.,색이 참 예뻐요.,
5,28834,5,GSSHOP,musi****님,2016.01.29.,가격 품질 모두 만족합니다.,가격 품질 모두 만족합니다.,
6,28834,5,GSSHOP,pink****님,2016.01.05.,서비스가 이래도 되나요.....♡아이브로우 카라와 매니큐어 하나 주문했을 뿐인데쇼핑백에,서비스가 이래도 되나요.....♡아이브로우 카라와 매니큐어 하나 주문했을 뿐인데쇼핑...,https://shopping-phinf.pstatic.net/20160106_2/...
7,28834,5,롯데아이몰,enab****님,2016.01.08.,맘에 들어요. 배송 빠르고 사은품도 감사.,맘에 들어요. 배송 빠르고 사은품도 감사.,
8,28834,5,롯데아이몰,fri2****님,2015.10.23.,아주 조아요. 배송도 빠르고요.,아주 조아요. 배송도 빠르고요.,
9,28834,5,롯데닷컴,leei****님,2015.10.10.,러블리로 사고 싶었지만 쉬크가 더 이쁘네여~색도...,러블리로 사고 싶었지만 쉬크가 더 이쁘네여~색도 예쁘고 좋으네여~,


In [224]:
import pickle

In [225]:
#with open('naver_reviews.plk','wb') as f:
#    pickle.dump(sample_reviews, f)

In [232]:
#item_d_info = pd.DataFrame(all_naver_iteminfos)
#item_d_info.to_csv('item_detail_naver.csv', index=False)